# Code Controlling the Quantum Computer

In [13]:
# whether to use the IBMQ backends (True)
# or a simulator (False)
USE_IBMQ = False

# IBMQ token to be used to access
# backends
IBMQ_TOKEN = 'TOKEN_HERE'

# IBMQ quantum computer to be used
IBMQ_QC_NAME = 'ibm_nairobi'

In [2]:
from qiskit import *
import pygame, sys, math, random

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [39]:
# run this to save token to disk
IBMQ.save_account(IBMQ_TOKEN)

configrc.store_credentials:WARNING:2022-12-16 03:15:41,879: Credentials already present. Set overwrite=True to overwrite.


## Generating the Quantum Circuit
This function creates the quantum circuit for this program. It consists of seven qubits and implements Grover's algorithm to solve a 3-variable boolean satisfiability problem. 

In [3]:
# in boolean_table
#   1 = false
#   2 = true
def generate_circuit(boolean_table):
    inputs = QuantumRegister(3, 'q')
    clauses = QuantumRegister(3, 'c_q')
    check = QuantumRegister(1, 'ch_q')
    classical = ClassicalRegister(3)
    circuit = QuantumCircuit(inputs, clauses, check, classical)
    
    circuit.h(inputs)
    circuit.h(check)
    circuit.z(check)
    circuit.barrier()
    
    add_oracle(circuit, boolean_table, inputs, clauses, check)
    circuit.barrier()
    add_diffusion(circuit, inputs)
    circuit.barrier()
    
    circuit.measure(inputs, classical)
    
    return circuit

### Generating the Oracle for Grover's Algorithm
This function creates the oracle for the quantum circuit based on the boolean formula in conjunctive normal form represented as `boolean_table`. 

In [4]:
def add_oracle(circuit, boolean_table, in_q, c_q, ch_q):
    add_half_oracle(circuit, boolean_table, in_q, c_q, ch_q)
    circuit.x(c_q)
    circuit.mct(c_q, ch_q[0])
    circuit.x(c_q)
    add_half_oracle(circuit, boolean_table, in_q, c_q, ch_q)

In [5]:
def add_half_oracle(circuit, boolean_table, in_q, c_q, ch_q):
    for i in range(len(boolean_table)):
        in_qubits = []
        not_qubits = []
        for j, val in enumerate(boolean_table[i]):
            if val == 1:
                not_qubits.append(in_q[j])
            elif val == 2:
                in_qubits.append(in_q[j])
        add_clause(circuit, c_q[i], in_qubits, not_qubits)

In [6]:
def add_clause(circuit, clause_qubit, input_qubits, not_qubits):
    if (len(input_qubits) + len(not_qubits) == 0):
        return
    if (len(input_qubits) > 0):
        circuit.x(input_qubits)
    circuit.mct(input_qubits + not_qubits, clause_qubit)
    if (len(input_qubits) > 0):
        circuit.x(input_qubits)

### Generating the Diffusion Operator for Grover's Algorithm
This function creates the diffusion operator for the quantum circuit. In short, this operator increases the probability that we will get the correct answer defined by the oracle.

In [7]:
def add_diffusion(circuit, qubits):
    circuit.h(qubits)
    circuit.x(qubits)
    circuit.h(qubits[-1])
    circuit.mct(qubits[:-1], qubits[-1])
    circuit.h(qubits[-1])
    circuit.x(qubits)
    circuit.h(qubits)

### Running the Quantum Circuit
These functions run the quantum circuit and processes the output of running the circuit to better suite the classical program.

In [8]:
def run_circuit(circuit):
    if USE_IBMQ:
        provider = IBMQ.load_account()
        backend = provider.get_backend(IBMQ_QC_NAME)
        trans_c = transpile(circuit, backend=backend)
        return backend.run(trans_c)
    
    backend = Aer.get_backend('aer_simulator')
    trans_c = transpile(circuit, backend=backend)
    job = backend.run(trans_c).result()
    return job
        

In [9]:
def get_solution(counts):
    minimum = min(counts.values())
    if (len(counts) != 8):
        minimum = 0
        
    maximum = max(counts.values())
    bound = maximum - minimum
    
    if (bound == 0):
        return counts
    
    threshold = minimum + (bound / 2)
    
    solution = dict(filter(lambda elem: elem[1] > threshold, counts.items()))
    
    return solution

# Code Running the Classical Program

In [14]:
WINDOW_SIZE = (800, 600)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
ORANGE = (255, 87, 51)
CIRCLE_RADIUS = 100.0
CIRCLE_CENTER = (400, 200)

max_time = 20.0
time = 0.0
iteration = 0
clock = pygame.time.Clock()
lost = False
buttons = []
button_states = []
mouse_down = False
score = 0
game_started = False
waiting = False

def play_game():
    global clock, old_ticks
    setup_game()
    window = pygame.display.set_mode(WINDOW_SIZE)
    old_ticks = pygame.time.get_ticks()
    
    while True:
        clock.tick(60)
        draw(window)
        update()
        
        old_ticks = pygame.time.get_ticks()
        
        pygame.display.update()
    return

def setup_game():
    global buttons, button_states, font, binaries, submit_button
    binaries = generate_random_binary()
    pygame.font.init()
    font = pygame.font.SysFont('Courier New', 50)
    submit_button = Button((330, 310), 140, 40)
    for i in range(9):
        row = math.floor(i / 3)
        col = math.floor(i % 3)
        but_pos = (row * 50 + 330, col * 50 + 360)
        buttons.append(Button(but_pos, 40, 40))
        button_states.append(False)
     
def update():
    global time, max_time, lost, binaries, mouse_down, game_started, waiting, current_job
    
    delta_time = (pygame.time.get_ticks() - old_ticks) / 1000.0
    
    for event in pygame.event.get():
        if not game_started:
            if event.type == pygame.MOUSEBUTTONDOWN or event.type == pygame.KEYDOWN:
                game_started = True
            return
        
        if waiting:
            if not current_job.done():
                return
            else:
                sol = get_solution(current_job.result().get_counts())
                handle_matching(sol)
                waiting = False
            return
        
        if event.type == pygame.QUIT:
            pygame.quit()
            pygame.display.quit()
            sys.exit()
            return
        elif event.type == pygame.MOUSEBUTTONDOWN:
            handle_click(event.pos)
        elif event.type == pygame.MOUSEBUTTONUP:
            mouse_down = False
    
    if lost or waiting:
        return
    
    if (time > max_time):
        lost = True
    else:
        time = time + delta_time
        
    return

def draw(window):
    global binaries, font, submit_button, score, game_started
    
    window.fill(BLACK) 
    
    if not game_started:
        draw_before_game(window)
        return
    
    if lost:
        draw_lose(window)
        return
    
    # draw the prompt text
    prompt = '  '.join(binaries)
    
    text_surface = font.render(prompt, False, WHITE)
    text_width, text_height = font.size(prompt)
    window.blit(text_surface, (400 - text_width / 2, 40))
    
    # draw the circle in the middle
    radius = (time / max_time) * CIRCLE_RADIUS
    pygame.draw.circle(window, ORANGE, CIRCLE_CENTER, CIRCLE_RADIUS, 1)
    pygame.draw.circle(window, ORANGE, CIRCLE_CENTER, radius)
    
    # draw the buttons
    for but in buttons:
        but.draw(window)
    
    submit_button.draw(window)
    
    # draw the score text
    text_surface = font.render('SCORE: ' + str(score), False, WHITE)
    text_width, text_height = font.size('SCORE: ' + str(score))
    window.blit(text_surface, (50, 600 - text_height - 10))
    
    if waiting:
        draw_waiting(window)

def draw_waiting(window):
    global font
    
    text_surface = font.render('WAIT', False, WHITE)
    text_width, text_height = font.size('WAIT')
    
    pygame.draw.polygon(window, BLACK, [(200, 250), (600, 250), (600, 350), (200, 350)])
    pygame.draw.polygon(window, WHITE, [(200, 250), (600, 250), (600, 350), (200, 350)], width=2)
    window.blit(text_surface, (400 - text_width/2, 300 - text_height/2))
        
def draw_before_game(window):
    global font
    
    text_surface = font.render('PRESS ANY KEY', False, WHITE)
    text_width, text_height = font.size('PRESS ANY KEY')
    window.blit(text_surface, (400 - text_width / 2, 220))
    
def draw_lose(window):
    global font, score
    
    text_surface = font.render('YOU LOST', False, WHITE)
    text_width, text_height = font.size('YOU LOST')
    window.blit(text_surface, (400 - text_width / 2, 200))
    
    text_surface = font.render('FINAL SCORE: ' + str(score), False, WHITE)
    text_width, text_height = font.size('FINAL SCORE: ' + str(score))
    window.blit(text_surface, (400 - text_width / 2, 300))

def handle_click(pos):
    global buttons, button_states, submit_button, mouse_down, waiting, current_job
    if mouse_down or waiting:
        return
    mouse_down = True
    for i, but in enumerate(buttons):
        if but.is_clicked(pos):
            but.set_filled(not but.filled)
            button_states[i] = not button_states[i]
            break
    if submit_button.is_clicked(pos):
        bool_table = [[0,0,0], [0,0,0], [0,0,0]]
        
        for i, but in enumerate(button_states):
            col = math.floor(i / 3)
            row = math.floor(i % 3)
            if but:
                bool_table[row][col] = 1
            else:
                bool_table[row][col] = 2
        
        circuit = generate_circuit(bool_table)
        print(circuit)
        job = run_circuit(circuit)
        
        if USE_IBMQ:
            waiting = True
            current_job = job
            return
        else:
            counts = job.get_counts()
        
        solution = get_solution(counts)
        handle_matching(solution)
        

def handle_matching(solution):
    global binaries, lost, score
    num_match = 0
    for b in binaries:
        if b[::-1] in solution:
            num_match = num_match + 1

    if num_match == 0:
        lost = True
    else:
        if num_match == 1:
            score = score + 100
        elif num_match == 2:
            score = score + 300
        elif num_match == 3:
            score = score + 500
        handle_score()
        
def handle_score():
    global time, max_time, binaries, iteration
    iteration = iteration + 1
    time = 0
    max_time = max_time / (0.1 * math.log(iteration + 1) + 1)
    binaries = generate_random_binary()
                

def generate_random_binary():
    result = []
    while len(result) < 3:
        num = random.randint(0, 7)
        num_str = format(num, '03b')
        if num_str in result:
            continue
        result.append(num_str)
    return result
    
class Button:
    # 40, 40
    def __init__(self, pos, width, height):
        self.pos = pos
        self.width = width
        self.height = height
        self.filled = False
    
    def is_clicked(self, pos):
        return (pos[0] >= self.pos[0] and pos[0] <= self.pos[0] + self.width and pos[1] >= self.pos[1] and pos[1] <= self.pos[1] + self.height)
    
    def set_filled(self, filled):
        self.filled = filled
    
    def draw(self, window):
        w = 0
        if not self.filled:
            w = 1
        pygame.draw.polygon(window, WHITE, [self.pos, (self.pos[0] + self.width, self.pos[1]), (self.pos[0]+self.width, self.pos[1]+self.height), (self.pos[0], self.pos[1]+self.height)], width=w)
        


play_game()

       ┌───┐      ░ ┌───┐     ┌───┐┌───┐     ┌───┐┌───┐     ┌───┐┌───┐     »
  q_0: ┤ H ├──────░─┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├┤ X ├─────»
       ├───┤      ░ └───┘  │  ├───┤└───┘  │  ├───┤├───┤  │  ├───┤└───┘     »
  q_1: ┤ H ├──────░────────■──┤ X ├───────■──┤ X ├┤ X ├──■──┤ X ├──────────»
       ├───┤      ░ ┌───┐  │  ├───┤┌───┐  │  ├───┤├───┤  │  ├───┤┌───┐     »
  q_2: ┤ H ├──────░─┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├┤ X ├─────»
       └───┘      ░ └───┘┌─┴─┐├───┤└───┘  │  └───┘└───┘  │  └───┘└───┘┌───┐»
c_q_0: ───────────░──────┤ X ├┤ X ├───────┼──────────────┼─────────■──┤ X ├»
                  ░      └───┘└───┘     ┌─┴─┐┌───┐       │         │  ├───┤»
c_q_1: ───────────░─────────────────────┤ X ├┤ X ├───────┼─────────■──┤ X ├»
                  ░                     └───┘└───┘     ┌─┴─┐┌───┐  │  ├───┤»
c_q_2: ───────────░────────────────────────────────────┤ X ├┤ X ├──■──┤ X ├»
       ┌───┐┌───┐ ░                                    └───┘└───┘┌─┴─┐└───┘»

/Users/jjkim/opt/anaconda3/lib/python3.9/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/jjkim/opt/anaconda3/lib/python3.9/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


       ┌───┐      ░           ┌───┐     ┌───┐┌───┐     ┌───┐               »
  q_0: ┤ H ├──────░────────■──┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├────────────■──»
       ├───┤      ░        │  └───┘  │  ├───┤└───┘  │  ├───┤            │  »
  q_1: ┤ H ├──────░────────■─────────■──┤ X ├───────■──┤ X ├────────────■──»
       ├───┤      ░ ┌───┐  │  ┌───┐  │  ├───┤       │  ├───┤┌───┐       │  »
  q_2: ┤ H ├──────░─┤ X ├──■──┤ X ├──■──┤ X ├───────■──┤ X ├┤ X ├───────■──»
       └───┘      ░ └───┘┌─┴─┐├───┤  │  └───┘       │  └───┘└───┘┌───┐┌─┴─┐»
c_q_0: ───────────░──────┤ X ├┤ X ├──┼──────────────┼─────────■──┤ X ├┤ X ├»
                  ░      └───┘└───┘┌─┴─┐┌───┐       │         │  ├───┤└───┘»
c_q_1: ───────────░────────────────┤ X ├┤ X ├───────┼─────────■──┤ X ├─────»
                  ░                └───┘└───┘     ┌─┴─┐┌───┐  │  ├───┤     »
c_q_2: ───────────░───────────────────────────────┤ X ├┤ X ├──■──┤ X ├─────»
       ┌───┐┌───┐ ░                               └───┘└───┘┌─┴─┐└───┘     »

SystemExit: 

/Users/jjkim/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
